<a href="https://colab.research.google.com/github/Hamza-Ali0237/PyTorch-Projects/blob/main/Intermediate/PyTorch-CelebA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi-Label Image Classification

## Setup & Configuration

In [ ]:
!pip install torchmetrics

In [ ]:
# Import Libraries
import torch
import torch.nn as nn
import torchvision as tv
import torchmetrics as tm
from torchvision import transforms
from torchvision.datasets import CelebA

import math
import random
import numpy as np
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
# Configuration Dictionary
cfg = {
    # Reproducibility
    "seed": 42,

    # Computing Device
    "device": "cuda" if torch.cuda.is_available() else "cpu",

    # Data Settings
    "root_dir": "./data",
    "image_size": 224,
    "batch_size": 32,
    "num_workers": 2,
    "pin_memory": True,
    "num_classes": 40,

    # Training Hyperparameters
    "epochs": 25,
    "lr": 1e-3,
    "weight_decay": 1e-4,
    "label_smoothing": 0.0,
}

In [ ]:
device = cfg["device"]

In [ ]:
# Set Seed For Reproducibilty
SEED = cfg["seed"]

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
  torch.cuda.manual_seed(SEED)
  torch.cuda.manua nl_seed_all(SEED)

# Ensure Deterministic Behaviour
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Transform & Load Data

In [ ]:
# Normalization Values
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
image_size = cfg["image_size"]

# Training Transforms (Augmentation+ Normalization)
train_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(image_size),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(
        brightness = 0.1, contrast = 0.1, saturation = 0.1, hue = 0.05
    ),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = mean, std = std
    )
])

# Validation & Test Transforms
val_test_transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.ToTensor(),
    transforms.Normalize(
        mean = mean, std = std
    )
])

In [ ]:
# Load Data
root = cfg["root_dir"]

train_dataset = CelebA(
    root = root, split = "train",
    transform = train_transforms, download = True
)

val_dataset = CelebA(
    root = root, split = "val",
    transform = val_test_transforms, download = True
)

test_dataset = CelebA(
    root = root, split = "test",
    transform = val_test_transforms, download = True
)

In [ ]:
# Define DataLoaders
batch_size = cfg["batch_size"]
num_workers = cfg["num_workers"]
pin_memory = cfg["pin_memory"]

train_loader = DataLoader(
    train_dataset, batch_size = batch_size,
    shuffle = True, num_workers = num_workers,
    pin_memory = pin_memory
)

val_loader = DataLoader(
    val_dataset, batch_size = batch_size,
    shuffle = False, num_workers = num_workers,
    pin_memory = pin_memory
)

test_loader = DataLoader(
    test_dataset, batch_size = batch_size,
    shuffle = False, num_workers = num_workers,
    pin_memory = pin_memory
)

## CNN Class

In [ ]:
class MultiLabelCNN(nn.Module):
  def __init__(self, num_classes, image_size):
        super(MultiLabelClass, self).__init__()

    self.features = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2)

        nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2)

        nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    # Dynamically Calculate Flattened Size
    with torch.no_grad():
      dummy = torch.zeros(1, 3, image_size, image_size)
      feat_dim = self.features(dummy).view(1, -1).shape[1]

    self.classifier = nn.Sequential(
        self.Flatten(),
        self.Linear(feat_dim, 512),
        self.ReLU(),
        self.Dropout(0.5),
        self.Linear(512, num_classes)
    )

  def forward(self, x):
    x = self.features(x)
    x = self.classifier(x)
    return x